In [1]:
import rascaline
rascaline._c_lib._get_library()

from copy import deepcopy

import numpy as np

from equisolve.numpy.scripts import MultiSpectraScript
from equisolve.numpy.models.linear_model import Ridge
from equisolve.numpy.preprocessing import StandardScaler
from equisolve.utils.convert import ase_to_tensormap

import ase.io


In [2]:
frames = ase.io.read("water_converted.extxyz", ":4") # CHANGE TO YOUR DATASET
y = ase_to_tensormap(frames, energy="TotEnergy", forces="force")

In [3]:
HYPERS = {
    "cutoff": 5.0,
    "max_radial": 8,
    "max_angular": 6,
    "atomic_gaussian_width": 0.25,
    "center_atom_weight": 1.0,
    "radial_basis":{"Gto": {"spline_accuracy": 1e-5}},
    "cutoff_function": {"ShiftedCosine": {"width": 0.2}},
    "radial_scaling": {"Willatt2018": {"exponent": 6, "rate": 3, "scale": 2.0}},
}

multi_spectra_hypers = {}
multi_spectra_hypers['Composition'] = {'per_structure': False}
multi_spectra_hypers['SoapRadialSpectrum'] = deepcopy(HYPERS)
multi_spectra_hypers['SoapRadialSpectrum'].pop('max_angular')
multi_spectra_hypers['SoapPowerSpectrum'] = deepcopy(HYPERS)

transformer = StandardScaler(parameter_keys=["values"])
estimator = Ridge(parameter_keys=["values", "positions"])
script = MultiSpectraScript(multi_spectra_hypers,
                            transformer_y=transformer,
                            estimator=estimator)


In [4]:
Xi = script.compute(systems=frames, gradients=["positions"])
script.fit(Xi, y, **{"estimator_kwargs": {"alpha": 1e-15}})

In [5]:
script.score(Xi, y)

1.2647021474138442

In [6]:
Xi = script.compute(systems=frames, gradients=["positions"])
y_pred = script.forward(Xi)


In [7]:
import pickle
with open("multi_spectra_script-thorben.pickle", "wb") as file:
    pickle.dump(script, file)

### Testing i-pi support

In [8]:
import pickle
with open("multi_spectra_script-thorben.pickle", "rb") as file:
    script = pickle.load(file)

In [9]:
# in ipi

import ase.io
import numpy as np

structure = ase.io.read("h5o2+.extxyz", "0")

Xi = script.compute(systems=structure, gradients=["positions"])
y_pred = script.forward(Xi) # implicitely done in score function
energy = y_pred.block().values[0][0]
forces = np.array(y_pred.block().gradient("positions").data.reshape(-1, 3))

print(energy)
print()
print(forces)

-30021.592706684605

[[-3.49000829 -0.66491428  0.03825963]
 [ 3.31762319  0.70152297  1.06043758]
 [-0.53010739 -0.29730224 -0.87017826]
 [ 0.06258619 -0.01292945 -0.39877504]
 [ 0.5565406  -0.76222816  0.55000543]
 [ 0.77346134  0.24635373 -0.68418884]
 [-0.69009565  0.78949744  0.30443949]]


In [ ]:
# install ipi from this fork and branch https://github.com/agoscinski/i-pi/tree/equihack
# pip install pip install git+https://github.com/agoscinski/i-pi/@equihack

In [11]:
from equisolve.numpy.scripts import GenericMDCalculator

from ipi.utils.mathtools import det_ut3x3
from ipi.utils.units import unit_to_internal, unit_to_user


md_calc = GenericMDCalculator("multi_spectra_script-thorben.pickle",
                    is_periodic=True,
                    structure_template="h5o2+.extxyz",
                    atomic_numbers=[1,8])

In [ ]:
# If it runs through you can go to i-pi side and follow the instruction here
# https://github.com/agoscinski/i-pi/tree/equihack/examples/equiscript
# using the multi_spectra_script-thorben.pickle